# Aula 1.2 — Introdução a Embeddings - Embeddings

# Instalação dos pacotes necessários / Configuração da máquina

In [ ]:
# Instalação das dependências localmente
# Se estiver rodando localmente, descomente a linha abaixo para instalar as dependências
# ! pip install -r requirements.txt

In [ ]:
# Se rodando no Google Colab, descomente a linha abaixo para montar o Google Drive
# from google.colab import drive
# drive.mount('/content/drive/')

In [ ]:
# Instalação das dependências no Google Colab
# Mude CAMINHO_PARA_REPO para o caminho correto do seu repositório no seu Google Drive
# ! pip install -r /content/drive/MyDrive/UFMS/Aulas/2025-2/TOPIA/repo/TopicosIA-2025-02/requirements.txt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import torch
import torch.nn as nn
import torch.optim as optim
from gensim.models import Word2Vec

## Onde estávamos: representações vetoriais tradicionais
- **One-hot (uma-quente):** vetor esparso com 1 na posição da palavra e 0 no resto.  
- **BoW (bag-of-words):** conta frequências de palavras, **ignora ordem**.  
- **n-gramas:** leva em conta sequências de tamanho *n* (ex.: bigramas), porém o espaço cresce muito.

**Consequência:** essas representações são úteis, mas **não codificam semelhança semântica** (ex.: *bom* vs *ótimo* são ortogonais em one-hot) e podem **falhar com negação** e **ordem**.

In [ ]:
import numpy as np
from collections import Counter

def cosine(u, v, eps=1e-9):
    u = np.array(u, dtype=float); v = np.array(v, dtype=float)
    return float(u.dot(v) / (np.linalg.norm(u)*np.linalg.norm(v) + eps))

# Vocabulário pequeno para ilustração
vocab = ["o","filme","é","não","bom","ótimo","atendeu","paciente","hospital"]
tok2idx = {t:i for i,t in enumerate(vocab)}

def bow_vector(tokens):
    x = np.zeros(len(vocab), dtype=float)
    for t in tokens:
        if t in tok2idx: x[tok2idx[t]] += 1
    return x

s1 = "o filme é bom".split()
s2 = "o filme é ótimo".split()
s3 = "o filme não é bom".split()

v1, v2, v3 = map(bow_vector, [s1,s2,s3])

print("cos( 'o filme é bom', 'o filme é ótimo'):", round(cosine(v1,v2),3))
print("cos( 'o filme é bom', 'o filme não é bom'):", round(cosine(v1,v3),3))

### Ordem importa, e BoW não vê
Com BoW, **“o paciente atendeu o hospital”** e **“o hospital atendeu o paciente”** são idênticas.  
Bigramas ajudam um pouco, mas o **número de features explode**.

In [ ]:
def ngrams(tokens, n=2):
    return ["_".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def vec_from_counts(counts, vocab):
    x = np.zeros(len(vocab), dtype=float)
    for i,t in enumerate(vocab):
        x[i] = counts.get(t, 0.0)
    return x

a = "o hospital atendeu o paciente".split()
b = "o paciente atendeu o hospital".split()

# BoW (unigramas)
u_vocab = vocab  # já definido
ua, ub = bow_vector(a), bow_vector(b)

# Bigramas
bigrams_a = Counter(ngrams(a,2))
bigrams_b = Counter(ngrams(b,2))
b_vocab = sorted(list(set(list(bigrams_a)+list(bigrams_b))))
xba = vec_from_counts(bigrams_a, b_vocab)
xbb = vec_from_counts(bigrams_b, b_vocab)

print("BoW (ignora ordem) — cos:", round(cosine(ua,ub),3), "(idênticas)")
print("Bigramas (vê ordem)  — cos:", round(cosine(xba,xbb),3), "(diferentes)")

### Por que n-gramas não escalam?
- Com vocabulário \(V\), o número potencial de **bigramas** é \(O(V^2)\); de **trigramas**, \(O(V^3)\).  
- Mesmo usando estruturas **esparsas**, o custo de memória/treino cresce rápido e muitos n-gramas raramente aparecem.

**Moral da história:** precisamos de **vetores densos** que **generalizem** além do que foi visto *literalmente*.

In [ ]:
def possible_ngrams(V, n):  # estimativa grosseira O(V^n)
    return V**n

for V in [5_000, 50_000]:
    print(f"\nVocabulário V={V:,}")
    for n in [1,2,3]:
        print(f"  n={n}: ~{possible_ngrams(V,n):,} features potenciais")

## O que são **embeddings**?
- Uma **função aprendida** \( f: \mathcal{X} \to \mathbb{R}^d \) que mapeia palavras, sub-palavras ou sentenças para **vetores densos** de baixa dimensão (\(d \ll |\mathcal{X}|\)).  
- **Princípio distribucional:** palavras com **contextos semelhantes** tendem a ter **vetores próximos** (alto cosseno).  
- **Geometria útil:** *direções* e *distâncias* capturam relações (semelhança, analogias).

Imagine num **mapa semântico**: *praia* fica perto de *mar*, longe de *hospital*; *médico* se aproxima de *saúde*, etc.

### Vantagens sobre BoW / n-gramas
- **Compartilham informação** entre palavras parecidas (ex.: *bom* e *ótimo*).  
- **Generalizam** para contextos não vistos literalmente.  
- Dimensão **controlada** (50–768+), em vez de crescer com \(V, V^2, V^3\).

## Como aprendemos embeddings? (alto nível)
- **Word2Vec (Skip-gram/CBOW):** aprende vetores prevendo **contexto** da palavra (ou vice-versa).  
- **GloVe:** fatoração explícita de coocorrências globais.  
- **fastText:** soma vetores de **sub-palavras** (n-gramas de caracteres) → lida melhor com OOV e morfologia.  
- **Embeddings contextuais (ELMo/BERT):** um vetor **por ocorrência**, dependente do **contexto** na sentença.

## Experimento inicial
Vamos “sentir” na prática as limitações que motivam embeddings.
1. **One-hot**: qualquer par de palavras diferentes tem cosseno \(0\).  
2. **BoW**: frases com **negação** ou **ordem invertida** podem parecer erroneamente parecidas.  
3. **Bigramas**: corrigem parcialmente, mas **não escalam**.

In [ ]:
# (1) One-hot não codifica semelhança: 'bom' ~ 'ótimo'?
def one_hot(word, vocab):
    x = np.zeros(len(vocab), dtype=float)
    if word in tok2idx: x[tok2idx[word]] = 1.0
    return x

oh_bom = one_hot("bom", vocab)
oh_otimo = one_hot("ótimo", vocab)

print("cos(one-hot('bom'), one-hot('ótimo')) =", cosine(oh_bom, oh_otimo), "(zero)")

In [ ]:
# (2) BoW erra na negação; (3) Bigramas ajudam um pouco
neg = "o filme não é bom".split()
pos = "o filme é bom".split()

bow_neg, bow_pos = bow_vector(neg), bow_vector(pos)
bi_neg = Counter(ngrams(neg,2)); bi_pos = Counter(ngrams(pos,2))
b_vocab = sorted(list(set(list(bi_neg)+list(bi_pos))))
xneg, xpos = vec_from_counts(bi_neg, b_vocab), vec_from_counts(bi_pos, b_vocab)

print("BoW  — cos(neg, pos) =", round(cosine(bow_neg, bow_pos),3))
print("Bi-gramas — cos(neg, pos) =", round(cosine(xneg, xpos),3))

## Breve história (do macro ao micro)
- **Anos 1990 — LSA/LSI:** fatoração (SVD) de matrizes termo-documento gera vetores densos.  
- **2003 — Neural Probabilistic LM (Bengio et al.):** primeiras redes neurais aprendendo embeddings junto com o LM.  
- **2013 — Word2Vec:** *skip-gram* / *CBOW* com *negative sampling* (rápido, popularizou embeddings).  
- **2014 — GloVe:** vetores a partir de coocorrências globais.  
- **2016/17 — fastText:** incorpora **sub-palavras** (morfologia e OOV).  
- **2018 — ELMo/BERT:** **embeddings contextuais** (um vetor por ocorrência).  
- **2019+ — Sentence-BERT e afins:** bons **embeddings de sentenças** para busca e recuperação.  
- **2020s — LLMs (GPT, etc.):** camadas internas geram representações ricas; APIs fornecem **embeddings universais**.

## Intuição
- **One-hot**: cada palavra → vetor esparso com 1 numa posição. Sem noção de semelhança.
- **Embeddings**: mapeiam cada palavra para um vetor **denso** de baixa dimensão (ex.: 50–300), aprendidos a partir de contexto.
- Ideia central (*Distribucional*): “Vocês conhecerão uma palavra pelo **contexto** que ela mantém”.
  
### Do ponto de vista computacional
- Vocabulário com \(V\) palavras. Dimensão do embedding \(d \ll V\).  
- Tabela \(E \in \mathbb{R}^{V \times d}\). A palavra \(w_i\) tem vetor \( \mathbf{e}_i = E[i] \).
- Similaridade por **cosseno**:
\[
\text{cos}(\mathbf{u}, \mathbf{v}) = \frac{\mathbf{u}\cdot\mathbf{v}}{\|\mathbf{u}\|\;\|\mathbf{v}\|}
\]

## De one-hot a vetores densos
Vamos construir vetores one-hot e comparar com embeddings aleatórios (apenas para enxergar a diferença de forma).

In [ ]:
# Vocabulário de exemplo 
vocab = ["brasil", "rio", "sao_paulo", "praia", "futebol", "selva", "amazonia", "cafe", "universidade", "hospital"]
idx = {w:i for i,w in enumerate(vocab)}
V = len(vocab)
d = 5  # dimensão dos embeddings "densos" para visualização

# One-hot
I = np.eye(V)

# Embeddings densos aleatórios (só para comparação visual)
np.random.seed(42)
E_rand = np.random.normal(scale=0.5, size=(V,d))

print("One-hot shape:", I.shape)
print("Embeddings densos shape:", E_rand.shape)

In [ ]:
# Visualização simples (PCA) dos embeddings aleatórios vs one-hot
def plot_points(X, labels, title):
    pca = PCA(n_components=2, random_state=0)
    X2 = pca.fit_transform(X)
    plt.figure(figsize=(6,5))
    plt.scatter(X2[:,0], X2[:,1], c="tab:blue")
    for i, txt in enumerate(labels):
        plt.annotate(txt, (X2[i,0]+0.01, X2[i,1]+0.01), fontsize=9)
    plt.title(title)
    plt.grid(alpha=0.3)
    plt.show()

plot_points(I, vocab, "One-hot (projeção PCA)")
plot_points(E_rand, vocab, "Vetores densos aleatórios (projeção PCA)")

**Observação:** One-hot não captura relação entre palavras. Já vetores densos *podem* capturar (se aprendidos de dados). Vamos treinar rapidamente um **Word2Vec** em um corpus pequeno de frases em PT-BR para demonstrar a ideia (não esperem qualidade de produção).  
*Siglas usadas*: OOV (*Out-Of-Vocabulary*), PCA (Análise de Componentes Principais), t-SNE (*t-distributed Stochastic Neighbor Embedding*).

In [ ]:
# 3) Treino rápido de Word2Vec com Gensim (toy)
corpus = [
    "o brasil gosta de futebol",
    "o rio tem praia bonita",
    "sao paulo tem universidade e hospital",
    "a amazonia tem selva e rios",
    "o brasil produz cafe",
    "o rio de janeiro tem futebol e praia",
    "sao paulo tem hospital famoso",
    "a universidade pesquisa inteligencia artificial",
    "cafe do brasil é famoso",
    "a selva da amazonia é densa",
    "o hospital do rio é universitario",
]

# Tokenização bem simples (minúsculas e split)
sentences = [s.lower().split() for s in corpus]

w2v = Word2Vec(
    sentences=sentences,
    vector_size=50,   # dimensão do embedding
    window=3,         # tamanho do contexto
    min_count=1,      # mantém todas as palavras do nosso mini-corpus
    sg=1,             # 1 = skip-gram, 0 = CBOW
    epochs=200,       # mais épocas pra compensar corpus pequeno
    seed=42
)

print("Palavras no vocabulário:", list(w2v.wv.key_to_index.keys()))

In [ ]:
# %%
# 4) Funções auxiliares: similaridade por cosseno e vizinhos mais próximos
from numpy.linalg import norm

def cosine(u, v, eps=1e-9):
    u = np.array(u); v = np.array(v)
    return float(u.dot(v) / (norm(u)*norm(v) + eps))

def nearest_neighbors(model, query, topk=5):
    if query not in model.wv:
        return []
    qv = model.wv[query]
    words = []
    sims = []
    for w in model.wv.index_to_key:
        if w == query: 
            continue
        s = cosine(qv, model.wv[w])
        words.append(w); sims.append(s)
    order = np.argsort(sims)[::-1][:topk]
    return [(words[i], sims[i]) for i in order]

for q in ["brasil", "rio", "universidade", "praia", "hospital", "amazonia"]:
    print(f"\nMais próximos de '{q}':")
    for w,s in nearest_neighbors(w2v, q, topk=5):
        print(f"  {w:15s}  cos={s: .3f}")

## Analogias 
Analogias famosas do tipo *rei − homem + mulher ≈ rainha* tendem a emergir em embeddings maiores e com muito dado.  
Com nosso corpus pequeno, veremos apenas um **exemplo didático**.

In [ ]:
def analogy(model, a, b, c, topk=5):
    # a : b :: c : ?
    if not all(w in model.wv for w in [a,b,c]):
        return []
    vec = model.wv[b] - model.wv[a] + model.wv[c]
    words = []
    sims = []
    for w in model.wv.index_to_key:
        if w in [a,b,c]: 
            continue
        s = cosine(vec, model.wv[w])
        words.append(w); sims.append(s)
    order = np.argsort(sims)[::-1][:topk]
    return [(words[i], sims[i]) for i in order]

print("Exemplo de analogia (qual palavra se aproxima de 'rio' - 'praia' + 'sao'):")
print(analogy(w2v, "praia", "rio", "sao"))

> **Nota:** Em corpus real (bilhões de tokens) as relações ficam bem mais nítidas. Aqui o objetivo é ver **a mecânica**.

## Embedding como camada de rede (PyTorch)
A camada `nn.Embedding(V, d)` implementa uma **tabela** onde cada índice retorna um vetor d-dimensional.  
Vamos treinar um mini-modelo para prever uma palavra do contexto (*CBOW* simplificado) apenas para “sentir” o gradiente atualizando a tabela.

In [ ]:
# Preparar dados (pares contexto->alvo) a partir do nosso corpus
word2idx = {w:i for i,w in enumerate(w2v.wv.index_to_key)}
idx2word = {i:w for w,i in word2idx.items()}
V = len(word2idx)
d = 32
window = 2

token_ids = [[word2idx[w] for w in s if w in word2idx] for s in sentences]

X, Y = [], []
for sent in token_ids:
    for i in range(len(sent)):
        left = max(0, i-window)
        right = min(len(sent), i+window+1)
        ctx = [sent[j] for j in range(left, right) if j != i]
        if not ctx:
            continue
        X.append(ctx)
        Y.append(sent[i])

# Pad de comprimento variável num jeito simples: média dos embeddings de contexto
# (para simplificar, dentro do forward vamos computar a média de embeddings)
X_lens = np.array([len(x) for x in X])

# Modelo CBOW simplificado
class CBOW(nn.Module):
    def __init__(self, vocab_size, d):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, d)
        self.lin = nn.Linear(d, vocab_size)
    def forward(self, ctx_batch):
        # ctx_batch: list of lists (índices)
        max_len = max(len(c) for c in ctx_batch)
        padded = []
        for c in ctx_batch:
            if len(c) < max_len:
                c = c + [0]*(max_len - len(c))
            padded.append(c)
        x = torch.tensor(padded, dtype=torch.long)
        E = self.emb(x)                   # [B, L, d]
        m = E.mean(dim=1)                 # [B, d]
        logits = self.lin(m)              # [B, V]
        return logits

model = CBOW(V, d)
opt = optim.Adam(model.parameters(), lr=0.05)
loss_fn = nn.CrossEntropyLoss()

# Treino rápido
def batchify(X, Y, bs=16):
    for i in range(0, len(X), bs):
        yield X[i:i+bs], Y[i:i+bs]

for epoch in range(150):
    total = 0.0
    for xb, yb in batchify(X, Y, bs=8):
        opt.zero_grad()
        logits = model(xb)
        y = torch.tensor(yb, dtype=torch.long)
        loss = loss_fn(logits, y)
        loss.backward()
        opt.step()
        total += float(loss)
    if (epoch+1) % 50 == 0:
        print(f"época {epoch+1:3d} | loss {total/len(X):.4f}")

# Extra: extrair embeddings aprendidos
E_cbow = model.emb.weight.detach().numpy()
print("Tabela de embeddings aprendida:", E_cbow.shape)

In [ ]:
# Inspecionar vizinhos no embedding do CBOW treinado
def nearest_from_matrix(E, word, topk=5):
    if word not in word2idx: 
        return []
    i = word2idx[word]
    q = E[i]
    sims = []
    words = []
    for j in range(V):
        if j == i: 
            continue
        s = cosine(q, E[j])
        sims.append(s); words.append(idx2word[j])
    order = np.argsort(sims)[::-1][:topk]
    return [(words[k], sims[k]) for k in order]

for q in ["brasil", "rio", "universidade", "praia"]:
    print(f"\nVizinhos (CBOW) de '{q}':")
    for w,s in nearest_from_matrix(E_cbow, q, 5):
        print(f"  {w:15s}  cos={s: .3f}")

## Sub-palavras e OOV
- *OOV* (*Out-Of-Vocabulary*): palavras não vistas no treino.  
- **fastText** usa **n-gramas de caracteres**: permite vetorizar “brasilão” mesmo sem tê-la visto, somando vetores de sub-palavras.  
Abaixo, um treino rápido **opcional** (pode demorar ~alguns segundos).

In [ ]:
try:
    from gensim.models.fasttext import FastText
    ft = FastText(sentences, vector_size=50, window=3, min_count=1, sg=1, epochs=100)
    for q in ["brasil", "brasilao", "universidade", "universitario"]:
        print(f"\nVizinhos (fastText) de '{q}':")
        if q in ft.wv:
            nnft = ft.wv.most_similar(q, topn=5)
            for w,s in nnft:
                print(f"  {w:15s}  cos={s: .3f}")
        else:
            print("  (palavra não está no vocabulário, mas fastText ainda gera vetor por sub-palavras)")
            vec = ft.wv.get_vector(q)  # ainda retorna vetor!
            # vamos medir distância desse vetor para algumas palavras conhecidas
            sims = [(w, cosine(vec, ft.wv[w])) for w in vocab if w in ft.wv]
            sims = sorted(sims, key=lambda x: x[1], reverse=True)[:5]
            for w,s in sims:
                print(f"  {w:15s}  cos={s: .3f}")
except Exception as e:
    print("fastText não disponível neste ambiente:", e)

## Embeddings de sentenças
Duas formas simples:
1. **Média** dos embeddings de palavras (rápido, baseline forte).  
2. **Modelo pré-treinado** (ex.: *Sentence-Transformers* multilíngue) — **opcional** (baixa o modelo).

In [ ]:
# Média de palavras (usando o Word2Vec treinado)
def sent_embed_mean(sent, model):
    toks = [w for w in sent.lower().split() if w in model.wv]
    if not toks:
        return np.zeros(model.vector_size)
    mat = np.stack([model.wv[w] for w in toks])
    return mat.mean(axis=0)

docs = [
    "praia lotada no rio de janeiro",
    "hospital universitario em sao paulo",
    "pesquisa em inteligencia artificial na universidade",
    "cafe do brasil é exportado",
    "passeio na amazonia e na selva"
]
D = np.stack([sent_embed_mean(s, w2v) for s in docs])

def search(query, k=3):
    qv = sent_embed_mean(query, w2v)
    scores = [cosine(qv, D[i]) for i in range(len(docs))]
    order = np.argsort(scores)[::-1][:k]
    return [(docs[i], scores[i]) for i in order]

for q in ["praia no rio", "universidade e hospital", "selva amazonia", "cafe brasileiro"]:
    print(f"\nConsulta: {q}")
    for d,s in search(q, 3):
        print(f"  → {d:70s}  cos={s:.3f}")

In [ ]:
# ]embeddings de sentenças com modelo pré-treinado (multilíngue)
# AVISO: baixa ~100MB na primeira vez.
from sentence_transformers import SentenceTransformer, util
try:
    model_st = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    E_docs = model_st.encode(docs, convert_to_tensor=True, normalize_embeddings=True)
    def search_st(query, k=3):
        q = model_st.encode([query], convert_to_tensor=True, normalize_embeddings=True)
        sims = util.cos_sim(q, E_docs).cpu().numpy().ravel()
        order = np.argsort(sims)[::-1][:k]
        return [(docs[i], float(sims[i])) for i in order]
    for q in ["praia no rio", "universidade e hospital", "selva amazonia", "cafe brasileiro"]:
        print(f"\n(Pré-treinado) Consulta: {q}")
        for d,s in search_st(q, 3):
            print(f"  → {d:70s}  cos={s:.3f}")
except Exception as e:
    print("Sentence-Transformers indisponível/opcional falhou:", e)

## Visualização (PCA e t-SNE)
Visualizações ajudam a intuir agrupamentos semânticos. Vamos projetar algumas palavras.

In [ ]:
words_to_plot = ["brasil","rio","sao","paulo","sao_paulo","praia","futebol","universidade","hospital","amazonia","selva","cafe"]
# normalizar nomes para o vocabulário do modelo w2v
words_to_plot = [w for w in words_to_plot if w in w2v.wv]

X = np.stack([w2v.wv[w] for w in words_to_plot])

# PCA
pca = PCA(n_components=2, random_state=0)
X2 = pca.fit_transform(X)

plt.figure(figsize=(6,5))
plt.scatter(X2[:,0], X2[:,1], c="tab:green")
for i,w in enumerate(words_to_plot):
    plt.annotate(w, (X2[i,0]+0.01, X2[i,1]+0.01), fontsize=9)
plt.title("Projeção PCA dos embeddings (Word2Vec)")
plt.grid(alpha=0.3)
plt.show()

# t-SNE (pode demorar um pouco com muitos pontos; aqui é pequeno)
tsne = TSNE(n_components=2, perplexity=5, learning_rate='auto', init='pca', random_state=0)
X3 = tsne.fit_transform(X)
plt.figure(figsize=(6,5))
plt.scatter(X3[:,0], X3[:,1], c="tab:purple")
for i,w in enumerate(words_to_plot):
    plt.annotate(w, (X3[i,0]+0.8, X3[i,1]+0.8), fontsize=9)
plt.title("Projeção t-SNE dos embeddings (Word2Vec)")
plt.grid(alpha=0.3)
plt.show()

## Miniaplicação: busca semântica interativa (baseline)
Dado um conjunto de frases/documentos, calculem embeddings (média de palavras) e retornem os mais parecidos para uma consulta.

In [ ]:
# Vocês podem alterar a lista 'docs2' e testar consultas.
docs2 = [
    "ingressos para jogo de futebol no maracana",
    "trilha na floresta da amazonia",
    "praias de copacabana e ipanema",
    "hospital publico em sao paulo",
    "pesquisa de ia na universidade federal",
    "cafeteria tradicional no centro"
]
E2 = np.stack([sent_embed_mean(s, w2v) for s in docs2])

def search_docs(query, docs_list, E, k=3):
    qv = sent_embed_mean(query, w2v)
    scores = [cosine(qv, E[i]) for i in range(len(docs_list))]
    order = np.argsort(scores)[::-1][:k]
    return [(docs_list[i], scores[i]) for i in order]

# Exemplo
for q in ["futebol no rio", "praia", "selva amazonia", "universidade ia", "cafe"]:
    print(f"\nConsulta: {q}")
    for d,s in search_docs(q, docs2, E2, 3):
        print(f"  → {d:60s}  cos={s:.3f}")

## Notas críticas (viés e limitações)
- **Viés**: embeddings refletem padrões do corpus. Se os dados têm estereótipos, o vetor também terá.  
- **Contexto**: embeddings “estáticos” (Word2Vec/GloVe) **não** diferenciam sentidos: *banco* (sentar) vs *banco* (financeiro).  
- **Solução moderna**: embeddings **contextuais** (BERT/Transformers) geram vetores **por ocorrência**.

## Resumo rápido
- Embeddings: mapeiam palavras/sentenças para vetores **densos** que capturam **similaridade semântica**.  
- É possível **treinar** (Word2Vec/CBOW/Skip-gram) ou **usar pré-treinados** (fastText, Sentence-Transformers).  
- Ferramentas: similaridade por cosseno, vizinhos, analogias, PCA/t-SNE.  
- Cuidados: viés, OOV, sentidos múltiplos.

# Exercícios

In [ ]:
# - Modifiquem o corpus adicionando 5–10 frases novas com tema de sua escolha (ex.: esportes, saúde, educação).
# - Re-treiem o Word2Vec (mudem window/vector_size/epochs) e observem:
#   a) Vizinhos mais próximos de 3 palavras.
#   b) Mudanças na visualização PCA.
# Dica: copiem e ajustem as células do treino e da visualização.


In [ ]:
# Implementem uma função 'most_similar_to_set(positivos, negativos, topk=5)' no embedding do w2v,
# que calcula: v = sum(positivos) - sum(negativos), e retorna vizinhos de v.
# Testem casos como: positivos=['rio','praia'], negativos=['selva'].

## Leituras e recursos adicionais
**Artigos / docs (abertos):**
- Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). *Efficient Estimation of Word Representations in Vector Space* (Word2Vec).  
- Pennington, J., Socher, R., & Manning, C. D. (2014). *GloVe: Global Vectors for Word Representation*.  
- Bojanowski, P., Grave, E., Joulin, A., & Mikolov, T. (2017). *Enriching Word Vectors with Subword Information* (fastText).  
- Reimers, N., & Gurevych, I. (2019). *Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks*.  
- Goldberg, Y. (2016). *A Primer on Neural Network Models for Natural Language Processing*.